In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
#print(os.listdir("../input"))

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [3]:
!python -m spacy download en_core_web_lg


    Linking successful
    /Users/matthewrallison/labenv/lib/python3.6/site-packages/en_core_web_lg
    -->
    /Users/matthewrallison/labenv/lib/python3.6/site-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [4]:
wines = pd.read_csv('/Users/matthewrallison/SAGE/prodigy_test2/wine-reviews/wine1.csv')

In [5]:
nlp = spacy.load('en_core_web_lg')

In [6]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [8]:
doc = nlp(wines["description"][0])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [9]:
review = str(" ".join([i.lemma_ for i in doc]))

In [10]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [27]:
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [12]:
tqdm.pandas()
wines["processed_description"] = wines["description"].progress_apply(spacy_tokenizer)

100%|██████████| 129971/129971 [06:40<00:00, 324.82it/s]


In [13]:
wines['description'][0]

"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [14]:
wines['processed_description'][0]

'aromas include tropical fruit broom brimstone dry herb palate overly expressive offer unripened apple citrus dry sage alongside brisk acidity'

In [21]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(wines["processed_description"])

In [32]:
data_vectorized[0]

<1x9710 sparse matrix of type '<class 'numpy.int64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [22]:
NUM_TOPICS = 3

In [23]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=1, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 1


In [24]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [25]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('flavor', 21044.852454989836), ('finish', 18010.231058813675), ('palate', 17510.486804853554), ('apple', 14118.11293596146), ('aroma', 13464.744024165831), ('white', 12538.492861058587), ('acidity', 11877.260183590422), ('fruit', 11331.514358097931), ('note', 11075.089283966603), ('citrus', 10130.53477476043)]
Topic 1:
[('wine', 62469.090974977946), ('fruit', 33851.21062905819), ('drink', 23233.578714212905), ('acidity', 19930.20145272179), ('flavor', 17430.55543722532), ('ripe', 17303.22503705121), ('texture', 15328.626858260126), ('rich', 14925.870937593612), ('age', 14038.407488125755), ('good', 13983.031974615924)]
Topic 2:
[('cherry', 33691.75710633292), ('flavor', 30026.088506855904), ('black', 24497.002046341473), ('aroma', 23484.704775947048), ('finish', 21468.94344757453), ('tannin', 21185.227014077107), ('palate', 20931.83644255391), ('fruit', 18894.905330146867), ('spice', 16387.665069280996), ('berry', 16093.338527009775)]


In [26]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                 x           y  topics  cluster       Freq
topic                                                     
2      -632.045471 -384.325073       1        1  42.172754
1     -1209.665405  212.164124       2        1  29.341543
0     -1437.453247 -586.322510       3        1  28.485703, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
9556  Default  82043.000000        wine  82043.000000  30.0000  30.0000
1524  Default  33047.000000      cherry  33047.000000  29.0000  29.0000
151   Default  13969.000000         age  13969.000000  28.0000  28.0000
8668  Default  15727.000000     texture  15727.000000  27.0000  27.0000
367   Default  15260.000000       apple  15260.000000  26.0000  26.0000
6036  Default  38386.000000      palate  38386.000000  25.0000  25.0000
9522  Default  13006.000000       white  13006.000000  24.0000  24.0000
2593  Default  34547.000000       drink  34547.000000  23.0000  23.0000
3192  Default  39928.000000      finish  39928.000000  22.0000  22.0000
422   Default  36758.000000       aroma  36758.000000  21.0000  21.0000
4753  Default  10176.000000       lemon  10176.000000  20.0000  20.0000
73    Default  35016.000000     acidity  35016.000000  19.0000  19.0000
7101  Default  18146.000000        rich  18146.000000  18.0000  18.0000
815   Default  29168.000000       black  29168.000000  17.0000  17.0000
6140  Default   9334.000000       peach   9334.000000  16.0000  16.0000
3458  Default  63698.000000       fruit  63698.000000  15.0000  15.0000
3465  Default   9168.000000      fruity   9168.000000  14.0000  14.0000
819   Default  14761.000000  blackberry  14761.000000  13.0000  13.0000
8529  Default  32147.000000      tannin  32147.000000  12.0000  12.0000
1616  Default  11725.000000      citrus  11725.000000  11.0000  11.0000
6149  Default   9042.000000        pear   9042.000000  10.0000  10.0000
1456  Default   8194.000000   character   8194.000000   9.0000   9.0000
5762  Default  25139.000000        note  25139.000000   8.0000   8.0000
1198  Default  11030.000000    cabernet  11030.000000   7.0000   7.0000
8242  Default  13047.000000   structure  13047.000000   6.0000   6.0000
7134  Default  27755.000000        ripe  27755.000000   5.0000   5.0000
4820  Default   6599.000000        lime   6599.000000   4.0000   4.0000
9644  Default   9676.000000        year   9676.000000   3.0000   3.0000
766   Default  18862.000000       berry  18862.000000   2.0000   2.0000
5904  Default   6204.000000      orange   6204.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
9688   Topic3   2693.774817       zesty   2730.461640   1.2422  -5.7663
6149   Topic3   8729.912983        pear   9042.484358   1.2206  -4.5905
4084   Topic3   3986.055458       honey   4080.636467   1.2323  -5.3745
1051   Topic3   2003.984443       brisk   2033.074945   1.2414  -6.0621
367    Topic3  14418.208146       apple  15260.320329   1.1990  -4.0888
5075   Topic3   1448.440477       mango   1473.533421   1.2386  -6.3868
1616   Topic3  10345.869854      citrus  11725.682313   1.1306  -4.4207
1176   Topic3   2480.809591      butter   2609.012701   1.2054  -5.8487
1468   Topic3   4369.221688  chardonnay   4893.501229   1.1424  -5.2827
2310   Topic3   2573.004907    delicate   2753.602241   1.1879  -5.8122
8905   Topic3   3477.171610    tropical   3855.653149   1.1524  -5.5110
3794   Topic3   7102.343106       green   9929.737270   0.9207  -4.7968
387    Topic3   3458.464466     apricot   4174.371645   1.0676  -5.5164
3432   Topic3  10034.592375       fresh  17505.615611   0.6993  -4.4512
6036   Topic3  17882.690459      palate  38386.366837   0.4919  -3.8734
3192   Topic3  18393.057298      finish  39928.750040   0.4806  -3.8453
5757   Topic3   8737.075344        nose  16851.666296   0.5989  -4.5897
5762   Topic3  11310.501854        note  25139.692071   0.4571

In [28]:
text = spacy_tokenizer("red")
print(lda.transform(vectorizer.transform([text])))

[[0.16670595 0.20759528 0.62569877]]


In [29]:
text = spacy_tokenizer("citrus")
print(lda.transform(vectorizer.transform([text])))


[[0.66318448 0.17014878 0.16666674]]
